# Create Inference Pipeline using Data Wrangler Flow & Autopilot Model

### Imports 

In [4]:
from sagemaker.pipeline import PipelineModel
from sagemaker.model import Model
from datetime import datetime
import sagemaker
import boto3
import os

### Essentials

In [5]:
session = sagemaker.Session()
s3_client = boto3.client('s3')
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d-%H-%M-%S')
iam_role = sagemaker.get_execution_role()
inference_bucket = session.default_bucket()

#### Push dataflow tar to S3

In [10]:
inference_flow_name = "loans_2022-12-22-04-37-36.tar.gz"

In [11]:
s3_client.upload_file(inference_flow_name, inference_bucket, f"data_wrangler_inference_flows/{inference_flow_name}", ExtraArgs={"ServerSideEncryption": "aws:kms"})
inference_flow_uri = f"s3://{inference_bucket}/data_wrangler_inference_flows/{inference_flow_name}"
print(f"Data Wrangler inference flow model URI: {inference_flow_uri}")

Data Wrangler inference flow model URI: s3://sagemaker-us-east-1-119174016168/data_wrangler_inference_flows/loans_2022-12-22-04-37-36.tar.gz


#### Get trained model location and image URI

In [12]:
automl_job_name = "zbu92oclw48b-AutoMLS-Px6ThX1MiP"
automl_job_desc = session.describe_auto_ml_job(automl_job_name)
best_inference_container = automl_job_desc["BestCandidate"]["InferenceContainers"][0]
algo_model_uri = best_inference_container["ModelDataUrl"]
print(f"Model URI: {algo_model_uri}")

Model URI: s3://sagemaker-us-east-1-119174016168/zbu92oclw48b-AutoMLS-Px6ThX1MiP/sagemaker-automl-candidates/model/WeightedEnsemble-L3-FULL-t3/model.tar.gz


In [24]:
best_inference_container

{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/autogluon-inference:0.4.3-cpu-py38-ubuntu20.04',
 'ModelDataUrl': 's3://sagemaker-us-east-1-119174016168/zbu92oclw48b-AutoMLS-Px6ThX1MiP/sagemaker-automl-candidates/model/WeightedEnsemble-L3-FULL-t3/model.tar.gz',
 'Environment': {'MODEL_NAME': 'WeightedEnsemble-L3-FULL',
  'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv',
  'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label',
  'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities,labels',
  'SAGEMAKER_PROGRAM': 'tabular_serve.py',
  'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}

In [13]:
dw_container_uri = "663277389841.dkr.ecr.us-east-1.amazonaws.com/sagemaker-data-wrangler-container:1.x"
algo_container_uri = best_inference_container["Image"]

### Create constituent models for the Inference Models 

In [14]:
pipeline_models = []
target_column_name = 'loan_status'
data_wrangler_model_name = f"DataWranglerInferencePipelineFlowModel-{timestamp}"

#### Re-create Data Wrangler model

In [15]:
data_wrangler_model = Model(image_uri=dw_container_uri, 
                            model_data=inference_flow_uri, 
                            role=iam_role, 
                            name=data_wrangler_model_name, 
                            sagemaker_session=session, 
                            env={"INFERENCE_TARGET_COLUMN_NAME": target_column_name})
pipeline_models.append(data_wrangler_model)

#### Re-create trained model (here Autopilot)

In [16]:
algo_model_name = f"DataWranglerInferencePipelineAlgoModel-{timestamp}"
algo_environment = best_inference_container["Environment"]

algo_model = Model(image_uri=algo_container_uri, 
                   model_data=algo_model_uri, 
                   role=iam_role, 
                   name=algo_model_name, 
                   sagemaker_session=session, 
                   env=algo_environment)

pipeline_models.append(algo_model)

In [18]:
inference_pipeline_model_name = f"DataWranglerInferencePipelineModel-{timestamp}"

inference_pipeline_model = PipelineModel(models=pipeline_models, 
                                         role=iam_role, 
                                         name=inference_pipeline_model_name, 
                                         sagemaker_session=session)
inference_pipeline_model.__dict__

{'models': [<sagemaker.model.Model at 0x7efee22dc110>,
 'role': 'arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628',
 'predictor_cls': None,
 'name': 'DataWranglerInferencePipelineModel-2022-12-23-02-25-44',
 'vpc_config': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7efee587b510>,
 'enable_network_isolation': False,
 'endpoint_name': None}

### Deploy the Pipeline model and create a real-time endpoint

In [19]:
instance_type = "ml.m5.xlarge"
instance_count = 2
endpoint_name = f"DataWranglerInferencePipelineEndpoint-{timestamp}"

In [20]:
inference_pipeline_model.deploy(instance_count, 
                                instance_type, 
                                endpoint_name=endpoint_name)

--------------------!

### Invoke the Inference Pipeline to get predictions

In [21]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name, 
                      sagemaker_session=session)

In [22]:
payload = '2500.0,60.0,15.27,c,car,1.0,30000.0'

In [23]:
prediction = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
print(f"The predicted target is: {prediction}")

The predicted target is: b'default\n'


The predicted target is: 'fully paid'